In [7]:
from bs4 import BeautifulSoup
import pathlib, os
import networkx as nx
from pyvis.network import Network
import requests
from collections import defaultdict
import matplotlib.colors as mcolors

Netzwerkvisualisierung der in den Briefen erwähnten Personen
Die beiden Graphen wurden dadurch realisiert, dass die Briefe in 2 verschiedenen Verzeichnissen gespeichert wurden & das Skript entsprechend über beide Verzeichnisse laufen gelassen wurde.
Entsprechend muss man ggf. im untersten Code-Abschnitt auch noch den Ziel-Pfad der HTML-Datei anpassen, damit die erste generierte Datei nicht überschrieben wird.

In [8]:
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data" / "annotated" / "Labowsky_Klibansky"

Für das Netzwerk und Operationen im Netzwerk wird die Python-Bibliothek [networkx](https://networkx.org/) verwendet. Dazu wird zunächst ein Graph-Objekt erstellt, das anschließend die Knoten und Kanten vorhält. 

In [3]:
G = nx.Graph()

Um ausgehend von der GND-ID einen Namen sowie ein Geburtsdatum zu erhalten, wird in dieser Funktion eine entsprechende Anfrage an Wikidata gestellt. Dabei wird die Anfrage mit allen IDs gleichzeitig ausgeführt, um die Anzahl der Anfragen an Wikidata zu minimieren. Wikidata hat ein Limit an Anfragen pro Minute, deswegen ist dieses Vorgehen unbedingt notwendig.
Name, Geburtsdatum, Todesdatum und Arbeitgeber werden zugehörig zu der GND-ID in einem Dictionary zurückgegeben.

In [9]:
def get_wikidata(gnd_ids):
    values = " ".join(f'"{gnd}"' for gnd in gnd_ids)
    query = f"""
    SELECT ?gnd ?person ?personLabel ?birthDate ?deathDate ?employerLabel WHERE {{
      VALUES ?gnd_id {{ {values} }}
      ?person wdt:P227 ?gnd_id.
      BIND(?gnd_id AS ?gnd)
      OPTIONAL {{ ?person wdt:P569 ?birthDate. }}
      OPTIONAL {{ ?person wdt:P570 ?deathDate. }}
      OPTIONAL {{ ?person wdt:P108 ?employer. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    url = 'https://query.wikidata.org/sparql'
    headers = {'Accept': 'application/sparql-results+json'}
    response = requests.get(url, params={'query': query}, headers=headers)
    data = response.json()

    temp = defaultdict(lambda: {"name": None, "birth": None, "death": None, "employers": set()})

    for row in data['results']['bindings']:
        gnd_key = 'gnd-' + row['gnd']['value']
        name = row.get('personLabel', {}).get('value')
        birth = row.get('birthDate', {}).get('value', None)
        death = row.get('deathDate', {}).get('value', None)
        employer = row.get('employerLabel', {}).get('value', None)

        temp[gnd_key]["name"] = name
        temp[gnd_key]["birth"] = birth
        temp[gnd_key]["death"] = death
        if employer:
            temp[gnd_key]["employers"].add(employer)

    results = {
        gnd: (
            data["name"], 
            data["birth"],
            data["death"], 
            sorted(data["employers"])
        )
        for gnd, data in temp.items()
    }

    return results

Funktion, um Farben im Ton shiften zu können

In [10]:
def adjust_color_brightness(hex_color, factor):
    """
    factor < 1 => dunkler, 
    factor > 1 => heller
    """
    rgb = mcolors.to_rgb(hex_color)
    adjusted = tuple(min(max(c * factor, 0), 1) for c in rgb)
    return mcolors.to_hex(adjusted)

In [11]:
for FILE_NAME in os.listdir(DATA_DIRECTORY):
    if not FILE_NAME.endswith(".xml"):
        continue

    with open(DATA_DIRECTORY / FILE_NAME, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "xml")

    pers_names = soup.find_all("persName")
    previous_ref = None

    for pers in pers_names:
        ref = pers.get("ref")
        name = pers.get_text(strip=True)

        if not ref:
            continue  

        if not G.has_node(ref):
            G.add_node(ref, label=ref, title=name, count=1, names=set())
        else:
            G.nodes[ref]["count"] += 1
            G.nodes[ref]["title"] += f", {name}"
    
        G.nodes[ref]["names"].add(name)

        if previous_ref and previous_ref != ref:
            G.add_edge(previous_ref, ref)

        previous_ref = ref

FileNotFoundError: [Errno 2] No such file or directory: '/home/peer/Uni/2025SoSe_Seminar-Philosophinnen-im-Exil-III/code/data/annotated/Labowsky_Klibansky'

In [ ]:
ids = []
for node_id in G.nodes:
    ids.append(G.nodes[node_id]["label"].replace('gnd-', ''))
person_data = get_wikidata(ids)

Für die Darstellung des Netzwerks in einer Visualisierung werden die Knoten von Bing, Labowsky und Klibansky rot eingefärbt, um sie schneller identifizieren zu können. Außerdem skalieren wir die Größe der Knoten anhand der Häufigkeit der Nennungen (insgesamt skaliert).

* Die restlichen Knoten werden eingefärbt nach dem "Employer", insoweit dieser bei Wikidata hinterlegt ist
* Außerdem werden Knoten, welche Personen mit einem Geburtsdatum vor 1860 haben, heller hinterlegt
* Knoten mit einem Todesdatum zwischen den ersten und letzten Brief-Datierungen werden dunkler hinterlegt. 
* Knoten, zu denen keine Daten von Wikidata geholt werden konnten, sind grau hinterlegt

In [ ]:
highlight_refs = {"gnd-1029912939", "gnd-118777378", "gnd-116183853"}
notable_employers = {'Kulturwissenschaftliche Bibliothek Warburg', 'University of London', 'Warburg Institute'}
for node_id in G.nodes:
    freq = G.nodes[node_id]["count"]
    G.nodes[node_id]["size"] = 10 + freq * 2
    if node_id in person_data:
        name = person_data[node_id][0]
        
        birth_date = person_data[node_id][1]
        birth_year = None
        if isinstance(birth_date, str) and len(birth_date) >= 4:
            try:
                birth_year = int(birth_date[:4])
            except ValueError:
                pass
            
        death_date = person_data[node_id][2]
        death_year = None
        if isinstance(death_date, str) and len(death_date) >= 4:
            try:
                death_year = int(death_date[:4])
            except ValueError:
                pass
            
        employers = person_data[node_id][3]
        if name  == "Q133761091": # Für diesen konkreten Fall funktioniert die Wikidata-Anfrage scheinbar nicht ganz korrekt - diese Lösung war auf die Schnelle einfacher, als die Anfrage umzubauen
            G.nodes[node_id]["title"] = "Kenneth Koelln"
            G.nodes[node_id]["label"] = "Kenneth Koelnn"
        else:
            G.nodes[node_id]["title"] = name
            G.nodes[node_id]["label"] = name
            
        if node_id in highlight_refs:
            G.nodes[node_id]["color"] = "#FF0000"
        else:
            if any(employer in notable_employers for employer in employers):
                G.nodes[node_id]["color"] = "#38D160"
            else:
                G.nodes[node_id]["color"] = "#97C2FC"
            
        if birth_year and birth_year < 1860:
            G.nodes[node_id]["color"] = adjust_color_brightness(G.nodes[node_id]["color"], 1.3)
            
        if death_year and 1930 < death_year < 1950:
            G.nodes[node_id]["color"] = adjust_color_brightness(G.nodes[node_id]["color"], 0.7)
            
        if birth_year:
            G.nodes[node_id]["title"] += f"\n({birth_year}-" 
            G.nodes[node_id]["label"] += f"\n({birth_year}-"
            
            if death_year:
                G.nodes[node_id]["title"] += f"{death_year})" 
                G.nodes[node_id]["label"] += f"{death_year})"
            else:
                G.nodes[node_id]["title"] += f")" 
                G.nodes[node_id]["label"] += f")"
        
    else:
        G.nodes[node_id]["label"] = G.nodes[node_id]["title"]
        G.nodes[node_id]["color"] = "#9F9F9F"
            
    del G.nodes[node_id]["names"]

Nachdem die Knoten vorverarbeitet wurden, erfolt hier ein Export in das `.gexf`-Format. Dieses Dateiformat lässt sich in [Gephi](https://gephi.org/) öffnen, einem WYSIWYG-Tool für die Analyse und Visualisierung von Netzwerken. 

Wurde hier drinnengelassen, falls andere dieses Skript zum Nachvollziehen nutzen wollen, wir haben mit Gephi aber nicht gearbeitet.

In [ ]:
nx.write_gexf(G, "brief-netzwerk.gexf")

Nachfolgend die eigentliche Visualisierung. Dazu wird die Bibliothek [pyvis](https://pyvis.readthedocs.io/en/latest/tutorial.html) verwendet und das Ergebnis in einer HTML-Datei gespeichert, die sich mit einem Browser öffnen lässt. 

In [ ]:
net = Network(height="1400px", width="100%", notebook=True,  bgcolor="#ffffff", font_color="black", cdn_resources='in_line')
net.from_nx(G)
net.force_atlas_2based()
net.show("brief-netzwerk-labowsky-klibansky.html")

brief-netzwerk-labowsky-klibansky.html
